# Distillation Q&A with Large Language Models
 
### Notebook details
 
This sample demonstrates how to train the selected student model using the teacher model, resulting in the creation of the distilled model.
 
We will use the Meta Llama 3.1 405B Instruct as the teacher model and the Meta Llama 3.1 8B Instruct as the student model.
 
**Note :**
 
- Distillation offering is only available in **West US 3** regions.
- The Meta Llama 3.1 405B Instruct model can only be used as a teacher model.
- The Meta Llama 3.1 8B Instruct can only be used as a student (target) model.
- Distllation is supported for Question and Answering (QnA) task, which is a standard task in benchmarking for Natural Language Understanding.

**Prerequisites :**
- Subscribe to the Meta Llama 3.1 405B Instruct and Meta Llama 3.1 8B Instruct, see [how to subscribe your project to the model offering in MS Learn](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-serverless?tabs=azure-ai-studio#subscribe-your-project-to-the-model-offering)

## Install the SDK v2

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity

%pip install mlflow
%pip install azureml-mlflow
%pip install -U datasets

## Import the required libraries

In [ ]:
# import required libraries

import base64
import json
import os
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Data

## Prerequisites

An AI Studio project in **West US 3** is required. Please follow [this](https://learn.microsoft.com/azure/ai-studio/how-to/fine-tune-model-llama?tabs=llama-two%2Cchatcompletion#prerequisites) document to setup your AI Studio project

## AI Studio project settings

Update following cell with the information of the AI Studio project just created.

In [ ]:
SUBSCRIPTION_ID = "<SUBSCRIPTION>"
RESOURCE_GROUP = "<RESOURCE_GROUP>"
AI_PROJECT_NAME = "<AI_PROJECT_NAME>"

## Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## Get handle to AI Studio project

In [ ]:
ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, AI_PROJECT_NAME)
ai_project = ml_client._workspaces.get(ml_client.workspace_name)
ai_project._workspace_id

## Pick a teacher model

We support **Meta-Llama-3.1-405B-Instruct** as the teacher model. 
### First deploy the teacher model in Azure AI Studio
* Deploy Meta Llama 3.1 405B Instruct as a serverless API - [link](https://aka.ms/meta-llama-3.1-405B-instruct-azure-ai-studio-docs#create-a-new-deployment)
* Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

Update the following cell with the information of the deployment you just created.

In [ ]:
# Llama-3-405B Teacher model endpoint name
# The serverless model name is the name found in ML Studio > Endpoints > Serverless endpoints > Model column
TEACHER_MODEL_NAME = "Meta-Llama-3.1-405B-Instruct"

# The serverless model endpoint name is the name found in ML Studio > Endpoints > Serverless endpoints > Name column
# The endpoint URL will be resolved from this name by the MLFlow component
TEACHER_MODEL_ENDPOINT_NAME = "Meta-Llama-3-1-405B-Instruct-vum"

## Pick a student model

We will use **Meta-Llama-3.1-8B-Instruct** as student model. We only support chat completion models that are available for PayGo finetuning in Azure AI Studio.

In [ ]:
STUDENT_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
STUDENT_MODEL_VERSION = 1

# retrieve student model from model registry
mlclient_azureml_meta = MLClient(credential, registry_name="azureml-meta")
student_model = mlclient_azureml_meta.models.get(
    STUDENT_MODEL_NAME, version=STUDENT_MODEL_VERSION
)

print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        student_model.name, student_model.version, student_model.id
    )
)

## Download the dataset from HuggingFace repo

For our example, we download and use the Common Sense Q&A dataset (https://huggingface.co/datasets/tau/commonsense_qa/) from HuggingFace.

The next few cells show basic data preparation for fine tuning:
* Visualize some data rows
* Preprocess the data and format it in required format. This is an important step for performing QandA as we add the required sequences/separators in the data. 
* While fintuning, text column is concatenated with ground_truth column to produce finetuning input. Hence, the data should be prepared such that `text + ground_truth` is your actual finetuning data.
* bos and eos tokens are added to the data by finetuning pipeline, you do not need to add it explicitly 

##### Here is an example of how the data should look like

text generation requires the training data to include at least 2 fields – one for ‘text’ and ‘ground_truth’ like in this example. The below examples are from Common Sense QandA dataset. 

Original dataset:

{'id': '1afa02df02c908a558b4036e80242fac', 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?', 'question_concept': 'revolving door', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['bank', 'library', 'department store', 'mall', 'new york']}, 'answerKey': 'A'}

{'id': 'a7ab086045575bb497933726e4e6ad28', 'question': 'What do people aim to do at work?', 'question_concept': 'people', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']}, 'answerKey': 'A'}

Formatted dataset the user might pass:

{"messages": [{"role": "system", "content": "You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'."}, {"role": "user", "content": "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?\nAnswer Choices:\n(A) bank\n(B) library\n(C) department store\n(D) mall\n(E) new york"}]}


{"messages": [{"role": "system", "content": "You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'."}, {"role": "user", "content": "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: What do people aim to do at work?\nAnswer Choices:\n(A) complete job\n(B) learn from each other\n(C) kill animals\n(D) wear hats\n(E) talk to each other"}]}


In [ ]:
# We can define train and test sample sizes here. Validation size is kept same as test sample size
import download_dataset

train_sample_size = 100
val_sample_size = 100

# Sample notebook using the dataset: https://huggingface.co/datasets/tau/commonsense_qa
dataset_name = "tau/commonsense_qa"
input_dataset = download_dataset.CQnAHuggingFaceInputDataset()

# Note: train_split_name and test_split_name can vary by dataset. They are passed as arguments in load_hf_dataset.
# If validation_split_name is None, the below function will split the train set to create the specified sized validation set.
train, val, _ = input_dataset.load_hf_dataset(
    dataset_name=dataset_name,
    train_sample_size=train_sample_size,
    val_sample_size=val_sample_size,
    train_split_name="train",
    val_split_name="validation",
)

print("Len of train data sample is " + str(len(train)))
print("Len of validation data sample is " + str(len(val)))

In [ ]:
try:
    path = "data"
    os.mkdir(path, 0o755)
except FileExistsError as err:
    print(f"Error: {err}")
except OSError as err:
    print(f"Error: {err}")

In [ ]:
train_data_path = "data/train_cqna_512.jsonl"
valid_data_path = "data/valid_cqna_256.jsonl"

for row in train:
    data = {"messages": []}
    data["messages"].append(
        {
            "role": "system",
            "content": "You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'.",
        }
    )
    question, choices, ans_key = row["question"], row["choices"], row["answerKey"]
    labels, choice_list = choices["label"], choices["text"]
    answer_choices = [
        "({}) {}".format(labels[i], choice_list[i]) for i in range(len(labels))
    ]
    answer_choices = "\n".join(answer_choices)
    data["messages"].append(
        {
            "role": "user",
            "content": "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: "
            + row["question"]
            + "\nAnswer Choices:\n"
            + answer_choices,
        }
    )
    with open(train_data_path, "a") as f:
        f.write(json.dumps(data) + "\n")

for row in val:
    data = {"messages": []}
    data["messages"].append(
        {
            "role": "system",
            "content": "You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'.",
        }
    )
    question, choices, ans_key = row["question"], row["choices"], row["answerKey"]
    labels, choice_list = choices["label"], choices["text"]
    answer_choices = [
        "({}) {}".format(labels[i], choice_list[i]) for i in range(len(labels))
    ]
    answer_choices = "\n".join(answer_choices)
    data["messages"].append(
        {
            "role": "user",
            "content": "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: "
            + row["question"]
            + "\nAnswer Choices:\n"
            + answer_choices,
        }
    )
    with open(valid_data_path, "a") as f:
        f.write(json.dumps(data) + "\n")

In [ ]:
import pandas as pd

val_df = pd.read_json("./data/valid_cqna_256.jsonl", lines=True)
val_df.head()

## Prepare data inputs



In [ ]:
train_data = None
train_data_name = "cqna_train_70"

train_data = ml_client.data.create_or_update(
    Data(
        path=train_data_path,
        type=AssetTypes.URI_FILE,
        description="Training dataset",
        name=train_data_name,
    )
)

train_data_asset_id = f"azureml://locations/{ai_project.location}/workspaces/{ai_project._workspace_id}/data/{train_data.name}/versions/{train_data.version}"
train_data_asset_id

In [ ]:
valid_data = None
valid_data_name = "cqna_valid_70"

valid_data = ml_client.data.create_or_update(
    Data(
        path=valid_data_path,
        type=AssetTypes.URI_FILE,
        description="validation dataset",
        name=valid_data_name,
    )
)

valid_data_asset_id = f"azureml://locations/{ai_project.location}/workspaces/{ai_project._workspace_id}/data/{valid_data.name}/versions/{valid_data.version}"
valid_data_asset_id

## Distillation strategy settings

We provide the option to leverage Chain of Thought (CoT) reasoning for distillation. CoT leverages step by step reasoning ability of the teacher model to generate more accurate labels.

In [ ]:
ENABLE_CHAIN_OF_THOUGHT = "true"

## Configure distillation
Define finetune parameters


Training parameters define the training aspects such as - 
1. the learning rate
2. the number of epochs to finetune
3. the data_generation_task_type , in this case its 'NLU_QA'
and so on

In [ ]:
mlclient_azureml = MLClient(credential, registry_name="azureml")

In [ ]:
distillation_pipeline_name = "oss_distillation_pipeline"
distillation_pipeline_component = mlclient_azureml.components.get(
    name=distillation_pipeline_name, label="latest"
)

In [ ]:
@pipeline
def distillation_pipeline(
    teacher_model_endpoint_name: str,
    enable_chain_of_thought: str,
    system_properties: str,
    input_finetune_model: Input,
    train_file_path: Input,
    validation_file_path: Input = None,
):
    oss_distillation = distillation_pipeline_component(
        teacher_model_endpoint_name=teacher_model_endpoint_name,
        enable_chain_of_thought=enable_chain_of_thought,
        train_file_path=train_file_path,
        validation_file_path=validation_file_path,
        # Finetune
        mlflow_model_path=input_finetune_model,
        model_asset_id=student_model.id,
        system_properties=system_properties,
        ## hyperparams
        learning_rate=0.00002,
        per_device_train_batch_size=1,
        num_train_epochs=3,
        data_generation_task_type="NLU_QA",
    )

    return {"output_model": oss_distillation.outputs.output_model}

In [ ]:
system_properties = {
    "finetune_oss": "True",
    "model_asset_id": student_model.id,
    "PipelineType": "Finetune",
    "azureml.PipelineType": "Finetune",
    "azureml.ModelName": student_model.name,
    "azureml.original_model_id": student_model.id,
    "azureml.trainingData.assetId": train_data_asset_id,
}

json_str = json.dumps(system_properties).replace(" ", "")

system_properties_b64_encoded = base64.b64encode(json_str.encode("utf-8")).decode(
    "utf-8"
)
print(f"System properties => {system_properties_b64_encoded}")

In [ ]:
train_file_path_input = Input(type="uri_file", path=train_data.path)
validation_file_path_input = Input(type="uri_file", path=valid_data.path)
input_finetune_model = Input(type="mlflow_model", path=student_model.id)
experiment_name = f"distillation-{TEACHER_MODEL_NAME}".replace(".", "-")

finetuning_job = distillation_pipeline(
    teacher_model_endpoint_name=TEACHER_MODEL_ENDPOINT_NAME,
    enable_chain_of_thought=ENABLE_CHAIN_OF_THOUGHT,
    system_properties=system_properties_b64_encoded,
    input_finetune_model=input_finetune_model,
    train_file_path=train_file_path_input,
    validation_file_path=validation_file_path_input,
)

finetuning_job.properties.update(system_properties)
print(f"job property: {finetuning_job.properties}")

# pipeline_job.identity = UserIdentityConfiguration()
finetuning_job.display_name = f"finetune-{student_model.name}"
finetuning_job.experiment_name = experiment_name
finetuning_job.settings.default_compute_type = "serverless"
finetuning_job.continue_on_step_failure = False
# pipeline_job.settings.force_rerun = True

## Submit pipeline job

In [ ]:
# Submit pipeline job to workspace
ft_job = ml_client.jobs.create_or_update(finetuning_job)
print(f"Submitted job, progress available at {ft_job.studio_url}")


# wait for the pipeline job to complete
ml_client.jobs.stream(ft_job.name)

## Consuming the distilled model

Once the above job completes, you should be able to deploy the model and use it for inferencing. To deploy this model, do the following:

* Go to AI Studio
* Navigate to the Fine-tuning tab on the left menu
* In the list of models you see, click on the model which got created from the distillation
* This should take you to the details page where you can see the model attributes and other details
* Click on the Deploy button on top of the page
* Follow the steps to deploy the model